In [0]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
val myDataSchema = StructType(
  Array(
    StructField("id",DecimalType(26,0),true),
    StructField("anio",IntegerType,true),
    StructField("mes",IntegerType,true),
    StructField("provincia",IntegerType,true),
    StructField("canton",IntegerType,true),
    StructField("area",StringType,true),
    StructField("genero",StringType,true),
    StructField("edad",IntegerType,true),
    StructField("estado_civil",StringType,true),
    StructField("nivel_de_instruccion",StringType,true),
    StructField("etnia",StringType,true),
    StructField("ingreso_laboral",IntegerType,true),
    StructField("condicion_actividad",StringType,true),
    StructField("sectorizacion",StringType,true),
    StructField("grupo_ocupacion",StringType,true),
    StructField("rama_actividad",StringType,true),
    StructField("factor_expansion",DoubleType,true)
  ));


In [1]:
val data = spark
	  .read
	  .option("inferSchema","true")
	  .option("header","true")
	  .option("delimiter","\t")
	  .csv("/home/davisalex22/Datos_ENEMDU_PEA_v2.csv")

In [2]:
println($"Total de filas: ${data.count}")

In [3]:
data.select("ingreso_laboral").summary().show()

In [4]:
// Verificacion si existen nulos
data.select("ingreso_laboral").groupBy("ingreso_laboral").count().sort($"count".desc).show(5)

Análisis General Basado en la columna ingreso_laboral

In [6]:
// Seleccionar datos que no son nulos
val dfIngresoLaboral = data.select("ingreso_laboral").where($"ingreso_laboral".isNotNull)

In [7]:
dfIngresoLaboral.count

Desviación estándar

In [9]:
val cantValoresEnDifRangos = scala.collection.mutable.ListBuffer[Long]()
val minValue = 0.0
val maxValue = 146030
val bins = 5.0
val range = (maxValue - minValue) / bins
var minCounter = minValue
var maxCounter = range
while(minCounter < maxValue){
    val valoresEnUnRango = dfIngresoLaboral.where($"ingreso_laboral".between(minCounter,maxCounter))
    cantValoresEnDifRangos.+=(valoresEnUnRango.count())
    minCounter = maxCounter
    maxCounter = maxCounter + range
}
println("Valores en diferentes rangos:")
cantValoresEnDifRangos.foreach(println)

In [10]:
val avg = dfIngresoLaboral.select(mean("ingreso_laboral"))
    .first()(0).asInstanceOf[Double]

Cálculo de la Desviación estándar

In [12]:
val stdDesv = dfIngresoLaboral.select(stddev("ingreso_laboral"))
    .first()(0).asInstanceOf[Double]

In [13]:
// Cálculo de límites
val inferior = avg - 3 * stdDesv
val superior = avg + 3 * stdDesv

 Filtrar

In [15]:
val valoresMenoresLInferior = dfIngresoLaboral.where($"ingreso_laboral" < inferior)
valoresMenoresLInferior.describe().show

In [16]:
val valoresMayoresLSuperior = dfIngresoLaboral.where($"ingreso_laboral" > superior)
valoresMayoresLSuperior.describe().show

In [17]:
valoresMayoresLSuperior.orderBy($"ingreso_laboral".desc).show

In [18]:
val dataSinOutliertsStdDev = dfIngresoLaboral.where($"ingreso_laboral" > inferior && $"ingreso_laboral" < superior)
dataSinOutliertsStdDev.select("ingreso_laboral").summary().show

 
Tukey o Boxplots

In [20]:
// Obtener cuartiles
data.select("ingreso_laboral").summary().show

In [21]:
// Cuartiles
val cuartiles = dfIngresoLaboral.stat.approxQuantile("ingreso_laboral", Array(0.25,0.50,0.75), 0.0)

In [22]:
val q1 = cuartiles(0)
val q2 = cuartiles(1)
val q3 = cuartiles(2)

In [23]:
// Calcular rango intercuartil
val iqr = q3 -q1


In [24]:
// Calcular los límites
val inferiorIQR = q1 - 1.5 * iqr
val superiorIQR = q3 + 1.5 * iqr


In [25]:
val valoresMenoresInferiorIqr = dfIngresoLaboral.where($"ingreso_laboral" < inferiorIQR)
valoresMenoresInferiorIqr.summary().show

In [26]:
val valoresMayoresSuperiorIqr = dfIngresoLaboral.where($"ingreso_laboral" > superiorIQR)
valoresMayoresSuperiorIqr.summary().show

In [27]:
= dfIngresoLaboral.where($"ingreso_laboral" > inferiorIQR && $"ingreso_laboral" < superiorIQR)
dataSinOutliertsIQR.select("ingreso_laboral").summary().show

 Comparando ambas aproximaciones

In [29]:
dataSinOutliertsStdDev.select("ingreso_laboral").summary().show

In [30]:
dataSinOutliertsIQR.select("ingreso_laboral").summary().show